In [1]:
!pip install SimpleITK
!pip install pyradiomics

     |████████████████████████████████| 47.4MB 91kB/s 
     |████████████████████████████████| 194kB 17.0MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 552kB 43.0MB/s 


In [2]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import time
import SimpleITK as sitk
from matplotlib.pyplot import cm
from skimage import measure
from skimage.measure import regionprops
from radiomics import featureextractor
import six
import csv

In [3]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Uni/Sem 8 - Spring 21/EBME 461 - Biomedical Image Processing and Analysis/Project')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Code

In [4]:
img = sitk.ReadImage(f'./ProstateX/ProstateX-0000/T2W.nii.gz')

In [5]:
# Reading the prostate mask and lesion annotation
pm = sitk.ReadImage("./ProstateX/ProstateX-0000/PM.nii.gz")
ls = sitk.ReadImage("./ProstateX/ProstateX-0000/LS1.nii.gz")

In [ ]:
# pyradiomics feature extractor requires some initial parameters to be set

params = {}
params['normalize'] = True
params['binWidth'] = 256
params['verbose'] = True

# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor(**params) 

# First disablign all features and enabling only glcm features
# extractor.disableAllFeatures()
# extractor.enableFeatureClassByName('glcm')

featureVector = extractor.execute(img, pm)
# print(list(featureVector.keys()))
# for featureName in featureVector.keys():
#   print(f"Computed {featureName}: {featureVector[featureName]}")

In [ ]:
rootdir = '/content/drive/MyDrive/Uni/Sem 8 - Spring 21/EBME 461 - Biomedical Image Processing and Analysis/Project/ProstateX'
img_dict = {}
ls_dict = {}
fieldnames = ['patient_id', 'ls/pm']
fieldnames.extend(list(featureVector.keys()))
print(fieldnames)
with open('features.csv', mode='w') as features:
    writer = csv.DictWriter(features, fieldnames = fieldnames)
    writer.writeheader()
    for subdir, dirs, files in os.walk(rootdir):
        subpath = subdir.split('/')
        if subpath[-1] not in 'ProstateX':
            patient_name = subpath[-1]
            img_dict[patient_name] = {}
            img = sitk.ReadImage(subdir + '/T2W.nii.gz')
            ls = sitk.ReadImage(subdir + '/LS1.nii.gz')
            pm = sitk.ReadImage(subdir+'/PM.nii.gz')
            adc = sitk.ReadImage(subdir+'/ADC.nii.gz')
            featVecLs = extractor.execute(img, ls)
            featVecPm = extractor.execute(img, pm)
            img_dict[patient_name]['image'] = img
            img_dict[patient_name]['lesion'] = ls
            img_dict[patient_name]['mask'] = pm
            img_dict[patient_name]['diff'] = adc
            featVecLs['patient_id'] = patient_name
            featVecLs['ls/pm'] = 'ls'
            featVecPm['patient_id'] = patient_name
            featVecPm['ls/pm'] = 'pm'
            writer.writerow(featVecLs)
            writer.writerow(featVecPm)